# <font color='#42AAFF'>Содержание</font><a id='toc0_'></a>    
1. [<font color='#42AAFF'>Импорт данных</font>](#toc1_)    
2. [<font color='#42AAFF'>Подготовка данных для тестирования классификатора</font>](#toc2_)    
3. [<font color='#42AAFF'>Определение метрики на тестовой выборке</font>](#toc3_)    
4. [<font color='#42AAFF'>Определение метрики на тестовой выборке c n_samples=500</font>](#toc4_)    
5. [<font color='#42AAFF'>Заключение</font>](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
import pandas as pd
import numpy as np
import time
from catboost import Pool, cv, CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from joblib import dump, load

# 1. <a id='toc1_'></a>[<font color='#42AAFF'>Импорт данных</font>](#toc0_)

Импортируем подготовленные признаки базовой и тестовой выборок:

In [33]:
features_base = pd.read_csv("features_base.csv", index_col=0).values
features_test = pd.read_csv("features_test.csv", index_col=0).values
target_test = pd.read_csv("target_test.csv", index_col=0).values

Импортируем индексы кандидатов выбранной конфигурации кластеров в терминах базовых индексов, расстояния между целевыми векторами и кандидатами и словарь соответствия строк базового набора и названия объекта:

In [34]:
idx = pd.read_csv("index_test1.csv", index_col=0).values
vecs = pd.read_csv("vecs_test1.csv", index_col=0).values
base_index = pd.read_csv("base_index_test1.csv", index_col=0,header=None).to_dict()[1]

# 2. <a id='toc2_'></a>[<font color='#42AAFF'>Подготовка данных для тестирования классификатора</font>](#toc0_)

Напишем функцию, позволяющую подготовить данные для тестирования бинарного классификатора catboost. Матрица признаков состоит из 72 элементов целевого вектора, 72 элеменетов вектора-кандидата и квадрата расстояния между ними. Целевой признак в случае совпадения целевого вектора и вектора кандидата равен 1,  в противном случае 0.

In [35]:
def prepare_data(idx,dist,x_base,base_index,x,y):
    idx_flat = idx.flatten()
    dist = dist.flatten().reshape(idx.shape[0]*idx.shape[1],1)
    data1 = x_base[idx_flat]
    data2 = []
    for ind in range(idx.shape[0]):    
        data2.extend(np.repeat([x[ind]],idx.shape[1],axis=0))
    data2 = np.array(data2)  
    X = np.hstack((data2,data1,dist))

    Y = []
    for target, el in zip(y[:,0].tolist(), idx.tolist()):
        for r in el:
            Y.append(int(target in base_index[r]))
    Y = np.array(Y)  
    return X,Y

Воспользуемся функцией подготовки данных для генерации признаков тренировочной выборки для обучения catboost:

In [36]:
%%time
x_test, y_test = prepare_data(idx,vecs,features_base,base_index,features_test,target_test)

Wall time: 17.9 s


# 3. <a id='toc3_'></a>[<font color='#42AAFF'>Определение метрики на тестовой выборке</font>](#toc0_)

Загрузим предварительно оптимизированную модель catboost:

In [37]:
model1 = load('model.joblib')

Подготовим функцию для расчёта метрики accuracy@5:

In [42]:
def calc_accuracy5(x,y,model,n_samples):
    proba = model.predict_proba(x)[:,1]
    proba = proba.reshape(int(x.shape[0]/n_samples),n_samples)
    y = np.array(y).reshape(int(x.shape[0]/n_samples),n_samples)
    acc5 = 0
    rec_ind = []
    for val,prob in zip(y,proba):
        acc5 += np.sum(val[np.argsort(prob)[-5:]])
        rec_ind.append(np.argsort(prob)[-5:])
    acc5 = acc5/len(y)
    return acc5, rec_ind

In [43]:
%%time
acc5, rec_idx = calc_accuracy5(x_test,y_test,model1,50)
acc5

Wall time: 1min 52s


0.73255

Получили значение метрики ~0.733 для n_samples = 50.

# 4. <a id='toc4_'></a>[<font color='#42AAFF'>Определение метрики на тестовой выборке c n_samples=500</font>](#toc0_)

In [14]:
idx = pd.read_csv("index_test2.csv", index_col=0).values
vecs = pd.read_csv("vecs_test2.csv", index_col=0).values
base_index = pd.read_csv("base_index_test2.csv", index_col=0,header=None).to_dict()[1]

In [11]:
idx.shape

(100000, 500)

In [21]:
 idx[1:100].flatten().shape[0]

49500

In [22]:
vecs[1:100].flatten().reshape(49500,1)

array([[29.768549],
       [31.133074],
       [36.491787],
       ...,
       [69.43155 ],
       [69.43515 ],
       [69.46259 ]])

In [31]:
def calc_accuracy5_big_data(idx,dist,x_base,base_index,x,y,model,k_folds=10):
    
    acc5 = 0
    #формируем индексы разбиений
    fold_size = idx.shape[0] // k_folds
    ind_folds = []
    for i in range(k_folds):
        ind_folds.append(np.arange(fold_size)+i*fold_size)

    for ind_fold in ind_folds:
        #формируем массив X для фолда
        idx_flat = idx[ind_fold].flatten()
        #print(idx_flat.shape)
        dist_flat = dist[ind_fold].flatten().reshape(idx_flat.shape[0],1)
        #print(dist.shape)
        #print(type(dist))
        data1 = x_base[idx_flat]
        data2 = []
        for ind in range(fold_size):    
            data2.extend(np.repeat([x[ind]],idx.shape[1],axis=0))
        data2 = np.array(data2)  
        X = np.hstack((data2,data1,dist_flat))
        #формируем массив Y для фолда
        Y = []
        for target, el in zip(y[ind_fold,0].tolist(), idx[ind_fold].tolist()):
            for r in el:
                Y.append(int(target in base_index[r]))          
        #считаем метрику
        proba = model.predict_proba(X)[:,1]
        proba = proba.reshape(fold_size,idx.shape[1])
        Y = np.array(Y).reshape(fold_size,idx.shape[1])
        
        for val,prob in zip(Y,proba):
            acc5 += np.sum(val[np.argsort(prob)[-5:]]) 
        print(acc5)
    acc5 = acc5/idx.shape[0]
    return acc5

In [32]:
calc_accuracy5_big_data(idx,vecs,features_base,base_index,features_test,target_test,model1)

7690
12308
17029
21651
26344
31068
35802
40459
45132
49935


0.49935

Как видно из полученного результата, классификатор обученный при одном дисбалансе классов даёт результаты сильно хуже при другом

# 5. <a id='toc5_'></a>[<font color='#42AAFF'>Заключение</font>](#toc0_)

1. Разработан 2х ступенчатый алгоритм рекомендательной системы, на первом этапе для отбора кандидатов применен faiss, а на втором полученная выборка использовалась для обучения бинарного классификатора catboost. После чего к результатам классификации применено ранжирование.
2. При количестве кандидатов faiss=50 получено значение целевой метрики на тестовой выборке accuracy@5 = 0.733